# Azure RAG POC - Document Search with LLM

This notebook demonstrates how to implement Retrieval Augmented Generation (RAG) using Azure services for efficient document search and Q&A.

## Architecture:
1. **Document Processing**: Load and chunk documents
2. **Embeddings**: Generate embeddings using Azure OpenAI
3. **Vector Store**: Store embeddings for similarity search (using FAISS for POC)
4. **RAG Pipeline**: Retrieve relevant docs and generate answers using Azure OpenAI

## Prerequisites:
- Azure OpenAI Service access
- API key and endpoint
- Deployed models for embeddings and completions


## 1. Installation and Setup


In [1]:
# Install required packages
%pip install openai==1.12.0 langchain==0.1.6 langchain-openai==0.0.5 \
    faiss-cpu==1.7.4 tiktoken==0.5.2 python-dotenv==1.0.0 \
    pypdf==4.0.1 requests beautifulsoup4


  Using cached langchain-0.1.6-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_openai-0.0.5-py3-none-any.whl.metadata (2.5 kB)
  Using cached tiktoken-0.5.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_community-0.0.37-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_community-0.0.36-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_community-0.0.35-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_community-0.0.34-py3-none-any.whl.metadata (8.5 kB)
  Using cached langchain_community-0.0.33-py3-none-any.whl.metadata (8.5 kB)
  Using cached langchai

## 2. Configuration


In [2]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Azure OpenAI Configuration
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT", "https://your-resource.openai.azure.com/")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY", "your-api-key-here")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")

# Model deployment names (these should match your Azure OpenAI deployments)
EMBEDDING_DEPLOYMENT = os.getenv("AZURE_EMBEDDING_DEPLOYMENT", "text-embedding-ada-002")
CHAT_DEPLOYMENT = os.getenv("AZURE_CHAT_DEPLOYMENT", "gpt-4")

print("✓ Configuration loaded")
print(f"  Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"  Embedding Model: {EMBEDDING_DEPLOYMENT}")
print(f"  Chat Model: {CHAT_DEPLOYMENT}")


✓ Configuration loaded
  Endpoint: https://ashwin-demo.openai.azure.com/
  Embedding Model: text-embedding-ada-002
  Chat Model: gpt-4.1


## 3. Initialize Azure OpenAI Client


In [3]:
from openai import AzureOpenAI

# Initialize Azure OpenAI client
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

print("✓ Azure OpenAI client initialized")


✓ Azure OpenAI client initialized


## 4. Document Loading and Processing

For this POC, we'll use sample publicly available documents. You can replace this with your own 30 documents.


In [4]:
# Load real documents from public sources (fetched from APIs)
import json

# Load documents fetched from Wikipedia, REST Countries, and Open Library APIs
with open('fetched_documents.json', 'r', encoding='utf-8') as f:
    sample_documents = json.load(f)

# Analyze the documents
sources = {}
for doc in sample_documents:
    source = doc.get('source', 'Unknown')
    sources[source] = sources.get(source, 0) + 1

print(f"✓ Loaded {len(sample_documents)} REAL documents from public API sources")
print("\nDocument sources breakdown:")
for source, count in sources.items():
    print(f"  - {source}: {count} documents")

print("\nSample document titles:")
for i, doc in enumerate(sample_documents[:10], 1):
    title = doc['title'][:60] + "..." if len(doc['title']) > 60 else doc['title']
    print(f"  {i}. {title}")
if len(sample_documents) > 10:
    print(f"  ... and {len(sample_documents) - 10} more")

print(f"\n🌐 These are REAL documents fetched from:")
print("   • Wikipedia (E-commerce topics)")
print("   • REST Countries API (Country profiles)")
print("   • Open Library (Business books)")


✓ Loaded 43 REAL documents from public API sources

Document sources breakdown:
  - Wikipedia: 18 documents
  - REST Countries API: 10 documents
  - Open Library: 15 documents

Sample document titles:
  1. E-commerce
  2. Customer service
  3. Online shopping
  4. Warranty
  5. Payment system
  6. Customer relationship management
  7. Consumer protection
  8. Supply chain
  9. Inventory
  10. Retail
  ... and 33 more

🌐 These are REAL documents fetched from:
   • Wikipedia (E-commerce topics)
   • REST Countries API (Country profiles)
   • Open Library (Business books)


## 5. Text Chunking

Split documents into smaller chunks for better retrieval performance.


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_documents(documents, chunk_size=500, chunk_overlap=50):
    """
    Split documents into smaller chunks for embedding.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    
    chunks = []
    for doc in documents:
        splits = text_splitter.split_text(doc['content'])
        for split in splits:
            chunks.append({
                'text': split,
                'title': doc['title'],
                'source': doc['title']
            })
    
    return chunks

# Chunk the documents
document_chunks = chunk_documents(sample_documents)

print(f"✓ Created {len(document_chunks)} chunks from {len(sample_documents)} documents")
print(f"\nExample chunk:")
print(f"  Title: {document_chunks[0]['title']}")
print(f"  Text: {document_chunks[0]['text'][:100]}...")


✓ Created 202 chunks from 43 documents

Example chunk:
  Title: E-commerce
  Text: E-commerce (electronic commerce) refers to commercial activities including the electronic buying or ...


## 6. Generate Embeddings using Azure OpenAI


In [6]:
import time
import numpy as np
from typing import List

def get_embeddings(texts: List[str], batch_size=16) -> List[List[float]]:
    """
    Generate embeddings for a list of texts using Azure OpenAI.
    Processes in batches to handle rate limits.
    """
    all_embeddings = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        
        try:
            response = client.embeddings.create(
                input=batch,
                model=EMBEDDING_DEPLOYMENT
            )
            
            batch_embeddings = [item.embedding for item in response.data]
            all_embeddings.extend(batch_embeddings)
            
            # Small delay to avoid rate limiting
            if i + batch_size < len(texts):
                time.sleep(0.5)
                
        except Exception as e:
            print(f"Error generating embeddings for batch {i}: {e}")
            raise
    
    return all_embeddings

# Generate embeddings for all chunks
print("Generating embeddings... This may take a few minutes.")
chunk_texts = [chunk['text'] for chunk in document_chunks]
embeddings = get_embeddings(chunk_texts)

print(f"✓ Generated {len(embeddings)} embeddings")
print(f"  Embedding dimension: {len(embeddings[0])}")


Generating embeddings... This may take a few minutes.
✓ Generated 202 embeddings
  Embedding dimension: 1536


## 7. Create Vector Store with FAISS


In [7]:
import faiss

class VectorStore:
    def __init__(self, embeddings, documents):
        """
        Initialize FAISS vector store for similarity search.
        """
        self.documents = documents
        self.embeddings = np.array(embeddings).astype('float32')
        
        # Create FAISS index
        dimension = self.embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.index.add(self.embeddings)
        
        print(f"✓ Vector store initialized with {self.index.ntotal} vectors")
    
    def search(self, query_embedding, k=5):
        """
        Search for k most similar documents to the query.
        """
        query_vector = np.array([query_embedding]).astype('float32')
        distances, indices = self.index.search(query_vector, k)
        
        results = []
        for idx, distance in zip(indices[0], distances[0]):
            results.append({
                'document': self.documents[idx],
                'distance': float(distance),
                'similarity': 1 / (1 + float(distance))  # Convert distance to similarity
            })
        
        return results

# Create vector store
vector_store = VectorStore(embeddings, document_chunks)


✓ Vector store initialized with 202 vectors


## 8. RAG Pipeline Implementation


In [ ]:
def rag_query(question: str, top_k: int = 3, temperature: float = 0.3) -> dict:
    question_embedding = get_embeddings([question])[0]
    retrieved_docs = vector_store.search(question_embedding, k=top_k)
    
    context = "\n\n".join([
        f"{doc['document']['title']}: {doc['document']['text']}"
        for doc in retrieved_docs
    ])
    
    system_prompt = "Answer based on the provided documents. If information isn't in the documents, say so."
    
    user_prompt = f"Documents:\n{context}\n\nQuestion: {question}"
    
    try:
        response = client.chat.completions.create(
            model=CHAT_DEPLOYMENT,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=temperature,
            max_tokens=300
        )
        answer = response.choices[0].message.content
    except Exception as e:
        answer = f"Error: {e}"
    
    return {
        'question': question,
        'answer': answer,
        'sources': [doc['document']['title'] for doc in retrieved_docs],
        'retrieved_docs': retrieved_docs
    }

print("✓ RAG ready")


✓ RAG pipeline ready


## 9. Test Queries


In [ ]:
def display_answer(result):
    print(f"\nQ: {result['question']}")
    print(f"\nA: {result['answer']}")
    print(f"\nSources: {', '.join(result['sources'])}")
    print("-" * 80)


In [ ]:
# Test with real questions
result = rag_query("What's e-commerce?")
display_answer(result)


QUESTION: What is your return policy and how long do I have to return an item?

ANSWER:
I don't have information about a specific retailer's return policy or the exact time frame for returns in my knowledge base. Return policies can vary depending on the retailer, and they usually require a receipt and that the merchandise is in resellable condition. Some retailers may offer exchanges or store credit if you don't have a receipt or if the initial refund period has passed. Please check with the specific retailer for their detailed return policy and allowed return period. (Document: Product return)

────────────────────────────────────────────────────────────────────────────────
SOURCES USED (5 documents):
  1. Product return
  2. Product return
  3. Product return
  4. Product return
  5. Product return

────────────────────────────────────────────────────────────────────────────────
RETRIEVED DOCUMENT SNIPPETS:

  [1] Product return (Similarity: 0.754)
      Many retailers will accept r

In [ ]:
result = rag_query("How many people live in Japan?")
display_answer(result)


QUESTION: How much does shipping cost and when will I receive my order?

ANSWER:
I don't have information about shipping costs or delivery times in my knowledge base.

────────────────────────────────────────────────────────────────────────────────
SOURCES USED (5 documents):
  1. Shopping cart software
  2. Customer service
  3. Product return
  4. Online shopping
  5. Product return

────────────────────────────────────────────────────────────────────────────────
RETRIEVED DOCUMENT SNIPPETS:

  [1] Shopping cart software (Similarity: 0.710)
      The software allows online shopping customers to accumulate a list of items for purchase. At the point of sale, the software typically calculates a to...

  [2] Customer service (Similarity: 0.695)
      Customer support...

  [3] Product return (Similarity: 0.689)
      .  In some cases, only exchanges or store credit are offered, again usually only without a receipt, or after an initial refund period has passed. Some...




In [ ]:
result = rag_query("What currency does Brazil use?")
display_answer(result)


QUESTION: What payment methods do you accept?

ANSWER:
I don't have information about that in my knowledge base.

────────────────────────────────────────────────────────────────────────────────
SOURCES USED (5 documents):
  1. Payment system
  2. Payment system
  3. Payment gateway
  4. Online shopping
  5. Payment gateway

────────────────────────────────────────────────────────────────────────────────
RETRIEVED DOCUMENT SNIPPETS:

  [1] Payment system (Similarity: 0.738)
      Modern payment systems use cash-substitutes as compared to traditional payment systems. This includes debit cards, credit cards, electronic funds tran...

  [2] Payment system (Similarity: 0.727)
      . Some payment systems also include credit mechanisms, which are essentially a different aspect of payment....

  [3] Payment gateway (Similarity: 0.723)
      A payment gateway facilitates a payment transaction by the transfer of information between a payment portal (such as a website, mobile phone or intera...

## 10. Custom Questions


In [ ]:
# Try your own questions
question = "How do payment systems work?"
result = rag_query(question)
display_answer(result)


## 11. Load Your Own Files


In [ ]:
# Load your own documents
def load_custom_docs(file_paths):
    docs = []
    for path in file_paths:
        with open(path, 'r') as f:
            content = f.read()
        docs.append({'title': os.path.basename(path), 'content': content})
    return docs

# Example: Load from a folder
# my_docs = load_custom_docs(['path/to/doc1.txt', 'path/to/doc2.txt'])
# chunks = chunk_documents(my_docs)
# embeddings = get_embeddings([c['text'] for c in chunks])
# vector_store = VectorStore(embeddings, chunks)


✓ Document loading function ready
